In [30]:
!apt-get update 

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1184 kB]
Fetched 1436 kB in

In [31]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (80.0.3987.163-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 91 not upgraded.


In [32]:
!pip install selenium

In [33]:
%env LANG=vi_VN.utf8

env: LANG=vi_VN.utf8


In [34]:
cd /content/drive/My\ Drive/ProjectII/data-crawling

/content/drive/My Drive/ProjectII/data-crawling


In [0]:
import numpy as np
import pandas as pd
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [0]:
# instantiate a webdriver
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-infobars")
options.add_argument("--mute-audio")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--lang=vi")

driver = webdriver.Chrome('chromedriver', options=options)
driver.maximize_window()

In [0]:
url = "https://www.facebook.com/"
email = "syncdriver99@gmail.com"
password = "ads354a68gw464as53f1a6w8a4w6g5awega13w5e"

In [0]:
def login(email, password):
    global driver
    driver.get(url)
    
    # filling the form
    driver.find_element_by_id("email").send_keys(email)
    driver.find_element_by_id("pass").send_keys(password)
    
    # clicking on the login button
    try:
        driver.find_element_by_id("loginbutton").click()
    except NoSuchElementException:
        # facebook new design
        driver.find_element_by_id("login").click()

In [0]:
login(email, password)

In [0]:
# dictionary of user ID
data = pd.read_csv("users.csv", dtype=str)
ids = data.iloc[:, 0].values
check = {}
for id in ids:
    check[id] = True

# urls list
urls = data.iloc[:, 2].values

In [0]:
def appendZero(num):
    if num > 1000:
        return ""
    if num < 10:
        return "000{}".format(num)
    elif num < 100:
        return "00{}".format(num)
    elif num < 1000:
        return "0{}".format(num)
    else:
        return "{}".format(num)

In [0]:
# get 30 posts in timeline
def get_timeline(driver):
    timeline = driver.find_element_by_xpath("//*[contains(@id, 'timeline_story_container')]")
    old_height = driver.execute_script("return document.scrollHeight")
    scroll = 0
    max_scroll = 50
    flag = False
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            WebDriverWait(driver, 15, 0.05).until(
                lambda driver: driver.execute_script("return document.body.scrollHeight") != old_height
            )
            old_height = driver.execute_script("return document.body.scrollHeight")
        except TimeoutException:
            break
        scroll += 1
        if timeline.find_elements_by_xpath("./div[35]"):
            break
        if scroll == 50 or flag:
            break
    raw_posts = []
    container = timeline.find_elements_by_xpath("./div[@class='_5pcb _4b0l _2q8l']")
    for item in container:
        if item.find_elements_by_xpath(".//ul[contains(@class, 'uiList uiCollapsedList uiCollapsedListHidden')]"):
            continue
        else:
            raw_posts.append(item)
    return raw_posts[:30]

In [0]:
def crawl_posts(driver, raw_posts):
    posts = []
    number = 0
    for raw_post in raw_posts:
        number += 1
        print("Post #%d" % number)
        
        post = {}

        # crawl post timestamp
        timestamp = raw_post.find_element_by_xpath(".//span[@class='timestampContent']").text
        post["timestamp"] = timestamp

        # crawl reactions

        # open popup window
        react_box = raw_post.find_elements_by_xpath(".//a[@data-testid='UFI2ReactionsCount/root']")

        if react_box:
            display = react_box[0]
            driver.execute_script("arguments[0].scrollIntoView();", raw_post)
            time.sleep(0.5)
            try:
                display.click()
            except ElementClickInterceptedException:
                loc = display.location
                driver.execute_script("window.scrollTo(0, arguments[0]);", loc["y"] - 200)
                display.click()
            WebDriverWait(driver, 10).until(
                lambda driver: driver.find_elements_by_xpath("//div[@class='_4-i2 _50f4']")
            )
            popup = driver.find_element_by_xpath("//div[@class='_4-i2 _50f4']")
            if popup.find_elements_by_xpath(".//a[text()='Xem thêm']"):
                count = 0
                while True:
                    count += 1
                    try:
                        WebDriverWait(popup, 1, 0.05).until(
                            EC.element_to_be_clickable((By.XPATH, ".//a[text()='Xem thêm']"))
                        )
                        expand = popup.find_element_by_xpath(".//a[text()='Xem thêm']")
                        try:
                            expand.click()
                        except ElementClickInterceptedException:
                            driver.execute_script("arguments[0].scrollIntoView(false);", expand)
                            expand.click()
                    except TimeoutException:
                        break
                    if count == 50:
                        break
            reactors = popup.find_elements_by_xpath(".//li[@class='_5i_q']")
            reacts = []
            for reactor in reactors:
                contain_ID_element = reactor.find_element_by_xpath(".//a[@data-gt]")
                data_gt = json.loads(contain_ID_element.get_attribute("data-gt"))
                uid = data_gt["engagement"]["eng_tid"]
                reacts.append(uid)
            post["reacts"] = reacts

            # close popup window
            close = popup.find_element_by_xpath("//a[@data-testid='reactions_profile_browser:close' and @title='Đóng']")
            try:
                close.click()
            except ElementNotInteractableException:
                WebDriverWait(close, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "."))
                )
                close.click()

        # crawl comments

        # expand all extensible points
        queue = raw_post.find_elements_by_xpath(".//a[@class='_4sxc _42ft']")
        while True:
            if not queue:
                break
            point = queue.pop(0)
            try:
                text = driver.execute_script("return arguments[0].innerHTML", point)
            except StaleElementReferenceException:
                continue
            parent = None
            if "bình luận" not in text:
                parent = point.find_element_by_xpath("./parent::div/parent::div/parent::li")

            old_height = driver.execute_script("return arguments[0].scrollHeight", raw_post)
            driver.execute_script("arguments[0].scrollIntoView(false)", point)
            time.sleep(0.5)
            try:
                point.click()
            except StaleElementReferenceException:
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                point.click()
            except ElementClickInterceptedException:
                loc = point.location
                driver.execute_script("window.scrollTo(0, arguments[0]);", loc["y"] - 200)
                point.click()
            WebDriverWait(raw_post, 10).until(
                lambda x: driver.execute_script("return arguments[0].scrollHeight", x) != old_height
            )
            if parent is not None:
                ex_points = parent.find_elements_by_xpath(".//a[@class='_4sxc _42ft']")
            else:
                ex_points = raw_post.find_elements_by_xpath(".//a[@class='_4sxc _42ft' and contains(text(), 'bình luận')]")
            queue += ex_points

        # get uid of commentors
        is_visited = {}
        cmts = raw_post.find_elements_by_xpath(".//a[@class='_6qw4']")
        comments = []
        for cmt in cmts:
            data_hovercard = cmt.get_attribute("data-hovercard")
            uid = data_hovercard.split("?")[1].split("&")[0].split("=")[1]
            if uid in is_visited:
                continue
            else:   
                is_visited[uid] = True
                comments.append(uid)

        post["comments"] = comments

        # add to posts list
        posts.append(post)
    
    return posts

In [0]:
prefix = "https://www.facebook.com"
chunk = {"start": 105, "end": 200}
for order in range(chunk["start"] + 1 , chunk["end"] + 1):
    print("User #%d\n==========" % order)
    url = urls[order - 1]
    driver.get(prefix + url)
    raw_posts = get_timeline(driver)
    posts = crawl_posts(driver, raw_posts)
    with open("posts/{0}_{1}.json".format(appendZero(order), ids[order - 1]), "w") as fp:
        json.dump(posts, fp, indent="  ", ensure_ascii=False)